In [1]:

import pandas as pd
import numpy as np
from scipy.sparse import csc_matrix, hstack
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Activation, Dense, Dropout,Input,Lambda
from sklearn.preprocessing import LabelBinarizer
from sklearn.datasets import fetch_20newsgroups



Using TensorFlow backend.


In [2]:
# For reproducibility
np.random.seed(1237)
 
#the data are downloaded to C:\Users\martinma\scikit_learn_data
train = fetch_20newsgroups(random_state=1,
                           subset='train',
                                          )
test = fetch_20newsgroups(random_state=1,
                          subset='test',
                           )

train_posts=train.data
train_tags =  train.target
test_posts=test.data
test_tags = test.target   


In [3]:
#read preprocessed data and remove the row index column
train_posts_embed=pd.read_csv("train_posts_embed.csv",sep="~").iloc[:,1:513]
test_posts_embed=pd.read_csv("test_posts_embed.csv",sep="~").iloc[:,1:513]
test_posts_embed=csc_matrix(test_posts_embed)
train_posts_embed=csc_matrix(train_posts_embed)

In [4]:


# 20 news groups
num_labels = 20
vocab_size = 100000
batch_size = 100
 
# define Tokenizer with Vocab Size
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train_posts)
 
x_train = csc_matrix(tokenizer.texts_to_matrix(train_posts, mode='tfidf'))
x_test = csc_matrix(tokenizer.texts_to_matrix(test_posts, mode='tfidf'))
 
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)


In [5]:
encoder = LabelBinarizer()
encoder.fit(train_tags)
y_train = encoder.transform(train_tags)
y_test = encoder.transform(test_tags)

In [6]:
embed_size = 512
category_counts=20

In [7]:
from sklearn.linear_model import SGDClassifier
svm=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3,max_iter=60, random_state=42)
svm.fit(x_train, train.target)
predicted = svm.predict(x_test)
np.mean(predicted == test.target)

0.8416091343600637

In [8]:
#Merge text embeded datasets and tfidf datasets
train_posts=hstack([train_posts_embed,x_train])
test_posts=hstack([test_posts_embed,x_test])

In [9]:
from sklearn.linear_model import SGDClassifier
svm=SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3,max_iter=60, random_state=42)
svm.fit(train_posts, train.target)
predicted = svm.predict(test_posts)
np.mean(predicted == test.target)

0.8388210302708444

In [10]:
#Merge text embeded datasets and tfidf datasets
train_posts=hstack([train_posts_embed,x_train]).todense()
test_posts=hstack([test_posts_embed,x_test]).todense()

In [11]:
model = Sequential()
model.add(Dense(256, input_shape=(train_posts.shape[1],)))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_labels))
model.add(Activation('softmax'))
model.summary()
 
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 256)               25731328  
_________________________________________________________________
activation_1 (Activation)    (None, 256)               0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 64)                16448     
_________________________________________________________________
activation_2 (Activation)    (None, 64)                0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 20)                1300      
__________

In [12]:
history = model.fit(train_posts, y_train,
                    steps_per_epoch=80,
                    epochs=1,
                    verbose=1)
                   # validation_steps=80,
                   # validation_split=0.2)
                   # validation_data= (test_posts, y_test))
                  

Epoch 1/1
80/80 [==============================] - 7716s 96s/step - loss: 0.1638 - acc: 0.9683


In [14]:
score = model.evaluate(test_posts, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])
 


7532/7532 [==============================] - 73s 10ms/step
Test accuracy: 0.8544875227322429
